In [1]:
!git clone https://github.com/adeepH/SPDRA-2021-SharedTask.git 

Cloning into 'SPDRA-2021-SharedTask'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 112 (delta 52), reused 6 (delta 2), pack-reused 0
Receiving objects: 100% (112/112), 4.47 MiB | 532.00 KiB/s, done.
Resolving deltas: 100% (52/52), done.


In [2]:
cd /content/SPDRA-2021-SharedTask/

/content/SPDRA-2021-SharedTask


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 1.1MB 13.9MB/s 
     |████████████████████████████████| 1.2MB 47.9MB/s 
     |████████████████████████████████| 3.0MB 54.8MB/s 
     |████████████████████████████████| 890kB 47.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=bd6ffa68d3fb1ac3bc8a8bbff23818ed43715528bad59605d9c539fa52e4da3f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
cd /content/SPDRA-2021-SharedTask/scripts/

/content/SPDRA-2021-SharedTask/scripts


In [5]:
import pandas as pd
import numpy as np
import torch
from dataset import *
from model import *
from utils import *

In [6]:
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased') 
device = 'cuda'
MAX_LEN = 128
BATCH_SIZE = 32

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
model = RFClassifier(7,'allenai/scibert_scivocab_uncased')
model.load_state_dict(torch.load('/content/drive/MyDrive/spdra2021/saved_models/scibert.bin'))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

RFClassifier(
  (auto): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)

In [9]:
test = pd.read_csv('/content/drive/MyDrive/spdra2021/Datasets/test.csv',delimiter=',',
                 header=None,names=['text']) 

In [10]:
!python dataset.py

In [13]:

class SDPRATestDataset(Dataset):
  def __init__(self,text,tokenizer,max_len):
    self.text = text 
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.text)
  
  def __getitem__(self,item):
    text = str(self.text[item]) 

    encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length = self.max_len,
        return_token_type_ids = False,
        padding = 'max_length',
        return_attention_mask= True,
        return_tensors='pt',
        truncation=True
    )

    return {
        'text' : text,
        'input_ids' : encoding['input_ids'].flatten(),
        'attention_mask' : encoding['attention_mask'].flatten() 

    }


In [14]:
def create_dataloader(df,tokenizer,max_len,batch_size):
  ds = SDPRATestDataset(
      text = df.text.to_numpy(), 
      tokenizer = tokenizer,
      max_len = max_len
  )

  return DataLoader(ds,
                    batch_size = batch_size,
                    shuffle = False,
                    num_workers=4)


In [15]:
test_data_loader = create_dataloader(test,tokenizer,MAX_LEN,BATCH_SIZE)

In [16]:
def get_test_predictions(model, data_loader):
  model = model.eval()
  sentence = []
  predictions = []
  prediction_probs = [] 
  with torch.no_grad():
    for d in data_loader:
      texts = d["text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device) 
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      probs = nn.functional.softmax(outputs,dim=1)
      sentence.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs) 
  predictions = torch.stack(predictions).cpu().detach().numpy().tolist()
  prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy().tolist() 
  return sentence, predictions, prediction_probs 

In [17]:
y_review_texts, y_pred, y_pred_probs = get_test_predictions(
  model,
  test_data_loader
)

In [18]:
class_name = ['CL' ,'CR' ,'DC' ,'DS' ,'LO' , 'NI' , 'SE' ]
k = pd.DataFrame(y_pred_probs,columns=class_name)
k['result'] = k.idxmax(axis = 1) 
k['result'] = k['result'].apply({'CL':0,'CR':1,'DC':2,
'DS':3,'LO':4, 'NI':5, 'SE':6}.get) 
k['abstract'] = y_review_texts

In [19]:
k.result.value_counts()

5    1199
0    1184
1    1122
3    1074
6     850
2     812
4     759
Name: result, dtype: int64

In [ ]:
cd /content/drive/MyDrive/spdra2021/pred_probs/

/content/drive/MyDrive/spdra2021/pred_probs


In [20]:
k['result'] = k['result'].apply({0:'CL', 1:'CR', 2:'DC',
3:'DS', 4:'LO', 5:'NI', 6:'SE' }.get)

In [ ]:
result = k['result'].to_numpy()
print(len(result))
np.savetxt("run1.txt", result, fmt = "%s")

7000


In [ ]:
from google.colab import files
files.download('run1.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>